In [3]:
! bash /content/AHSG-InterIIT/src/setup-utils.sh
import sys  
sys.path.insert(0, '/content/AHSG-InterIIT/src')

     |████████████████████████████████| 1.0MB 20.5MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-cp37-none-any.whl size=993460 sha256=87b681ab971851861b8861d4a295a6a3dcaf28a0ac6556d744739f46c0269707
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
  Created wheel for syntok: filename=syntok-1.3.1-cp37-none-any.whl size=20919 sha256=38cba1f363799c3065b18b1aaf1f080753a2ac79bab4e5416ae60a81ecf1c5b3
  Stored in directory: /root/.cache/pip/wheels/51/c6/a4/be1920586c49469846bcd2888200bdecfe109ec421dab9be2d
Successfully built syntok
     |████████████████████████████████| 2.0MB 23.0MB/s 
     |████████████████████████████████| 890kB 53.7MB/s 
     |████████████████████████████████| 3.2MB 51.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=

In [4]:
%load_ext autoreload
%autoreload 2
%cd /content/AHSG-InterIIT

/content/AHSG-InterIIT


In [5]:
import pandas as pd
import brands,utils
import Article_Binary_Classifier_Inference
import Tweet_Binary_Classifier_Inference
import headline_generation
import sentiment_inference
import time

... OK (Got response in 0.13 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [7]:
!wget -O models/tweet_vect.pkl "https://drive.google.com/uc?export=download&id=1-BDg4YIu-CNHCwBUMhyXkEsun3M0Y3wQ"
!wget -O models/tweet_classf.pkl "https://drive.google.com/uc?export=download&id=1YsQO7LP5ezvDeh9MUlW6HRwKvWkXDYF6"
!wget -O models/article_vect.pkl "https://drive.google.com/uc?export=download&id=174SJ90A4mBpPa1gQIylS63k4e5kE41Wt"
!wget -O models/article_classf.pkl "https://drive.google.com/uc?export=download&id=1LqlpZNgho3unOhAfjdai6GQInBkjX7VJ"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-3OLtOIbJ6rhYpmbGdpaUuvNHyttqPWq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-3OLtOIbJ6rhYpmbGdpaUuvNHyttqPWq" -O /content/AHSG-InterIIT/models/T5-headline.pth && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1uRzeh66xQnH-dIkzISRL1c1spQnTG-uA' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1uRzeh66xQnH-dIkzISRL1c1spQnTG-uA" -O /content/AHSG-InterIIT/models/mbert-for-sentiment.pth && rm -rf /tmp/cookies.txt
!wget -O models/regression.pkl "https://drive.google.com/uc?export=download&id=1Hdck45_FQpt7D7lrEs3IvQ-lAqKeMDIX"
!wget -O models/tfidf.pkl "https://drive.google.com/uc?export=download&id=1jsN5kaj3tcCsxNEiF8DGLkmjwwFdG9IJ"

--2021-03-23 22:55:22--  https://drive.google.com/uc?export=download&id=1-BDg4YIu-CNHCwBUMhyXkEsun3M0Y3wQ
Resolving drive.google.com (drive.google.com)... 172.217.8.14, 2607:f8b0:4004:803::200e
Connecting to drive.google.com (drive.google.com)|172.217.8.14|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cu21g136epabqiqbpcan5sbr0de7olaq/1616540100000/02395324421011568547/*/1-BDg4YIu-CNHCwBUMhyXkEsun3M0Y3wQ?e=download [following]
--2021-03-23 22:55:23--  https://doc-0k-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cu21g136epabqiqbpcan5sbr0de7olaq/1616540100000/02395324421011568547/*/1-BDg4YIu-CNHCwBUMhyXkEsun3M0Y3wQ?e=download
Resolving doc-0k-5g-docs.googleusercontent.com (doc-0k-5g-docs.googleusercontent.com)... 172.217.2.97, 2607:f8b0:4004:80a::2001
Connecting to doc-0k-5g-docs.googleusercontent.com (doc-0k-5g-docs.google

In [133]:
df = pd.read_excel('/content/evaluation_data.xlsx')
df.head()

,Text_ID,Text
0,article_4001,Market Highlights\n\nAn automotive constant ve...
1,article_4002,Overview:\n\nBe it as a means for food or as p...
2,article_4003,"GenScript USA Inc. (U.S.), Horizon Discovery G..."
3,article_4004,New Delhi: Prime Minister Narendra Modi spoke ...
4,article_4005,"New Delhi, Feb 1 (PTI) The Delhi High Court Mo..."


In [134]:
df_articles = df[df['Text_ID'].str.contains('article')].copy().reset_index(drop=True)
df_tweets = df[df['Text_ID'].str.contains('tweet')].copy().reset_index(drop=True)

In [135]:
df_articles.head()

,Text_ID,Text
0,article_4001,Market Highlights\n\nAn automotive constant ve...
1,article_4002,Overview:\n\nBe it as a means for food or as p...
2,article_4003,"GenScript USA Inc. (U.S.), Horizon Discovery G..."
3,article_4004,New Delhi: Prime Minister Narendra Modi spoke ...
4,article_4005,"New Delhi, Feb 1 (PTI) The Delhi High Court Mo..."


In [136]:
df_tweets.head()

,Text_ID,Text
0,tweet_4001,@LloydsBank how do I report a scam on my accou...
1,tweet_4002,RT @IAmErAijaz: I have been denied security in...
2,tweet_4003,@FPLProp @Gooner_fpl I think there's a risk Sa...
3,tweet_4004,RT @brfootball: OFFICIAL: Bayern Munich loan d...
4,tweet_4005,"RT @TimInHonolulu: 5. So far, three 2016 cycle..."


In [137]:
start = time.time()
df_articles['Text'] = utils.clean_articles(df_articles['Text'].values)
df_tweets['Text'] = utils.clean_tweets(df_tweets['Text'].values, remove_emoji=False)
df_articles['brands'] = brands.get_brands(df_articles['Text'].values)
df_articles['num_brands'] = df_articles['brands'].apply(lambda x: len(x))
df_tweets['brands'] = brands.get_brands(df_tweets['Text'].values)
df_tweets['num_brands'] = df_tweets['brands'].apply(lambda x: len(x))
end = time.time()
PREPROCESS_TIME = end - start
print(PREPROCESS_TIME)

100%|██████████| 210/210 [00:00<00:00, 6016.05it/s]


1.066063404083252


In [138]:
start = time.time()
df_tweets = Tweet_Binary_Classifier_Inference.mobile_tech_binary_classifier(df_tweets)
df_articles = Article_Binary_Classifier_Inference.mobile_tech_binary_classifier(df_articles)
end = time.time()
BINARY_CLASSIFICATION_TIME = end-start
print(BINARY_CLASSIFICATION_TIME)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid res

0.20062828063964844


In [139]:
df_tweets['Mobile_Tech'].value_counts()

0    148
1     62
Name: Mobile_Tech, dtype: int64

In [140]:
df_articles['Mobile_Tech'].value_counts()

0    142
1     71
Name: Mobile_Tech, dtype: int64

In [141]:
df_articles_mob = df_articles[df_articles['Mobile_Tech']==1].copy().reset_index(drop=True)
df_tweets_mob = df_tweets[df_tweets['Mobile_Tech']==1].copy().reset_index(drop=True)

In [32]:
len(df_articles_mob), len(df_tweets_mob)

(71, 62)

In [33]:
start = time.time()
df_articles_mob['lang'] = utils.detect_lang(df_articles_mob['Text'].values)
df_tweets_mob['lang'] = utils.detect_lang(df_tweets_mob['Text'].values)
end = time.time()
LANGUAGE_DETECTION_TIME = end - start
print(LANGUAGE_DETECTION_TIME)


4.468086242675781


In [34]:
df_articles_mob['lang'].value_counts()

en      45
hi      24
hing     2
Name: lang, dtype: int64

In [35]:
df_tweets_mob['lang'].value_counts()

en      32
hi      18
hing    12
Name: lang, dtype: int64

In [36]:
start = time.time()
df_articles_mob['Text'] = brands.replace_hin_to_eng(df_articles_mob['Text'].values)
df_tweets_mob['Text'] = brands.replace_hin_to_eng(df_tweets_mob['Text'].values)
df_articles_mob['brands'] = brands.get_brands(df_articles_mob['Text'].values)
df_articles_mob['num_brands'] = df_articles_mob['brands'].apply(lambda x: len(x))
df_tweets_mob['brands'] = brands.get_brands(df_tweets_mob['Text'].values)
df_tweets_mob['num_brands'] = df_tweets_mob['brands'].apply(lambda x: len(x))
end = time.time()
PREPROCESS2_TIME = end - start
print(PREPROCESS2_TIME)

100%|██████████| 62/62 [00:00<00:00, 6205.48it/s]


0.24834942817687988


In [37]:
start = time.time()
# Hindi translation using Free Google API
df_articles_mob.loc[df_articles_mob['lang']=='hi','Text'] = utils.translate(df_articles_mob[df_articles_mob['lang']=='hi']['Text'].values)[0]

# Truncating Hinglish Articles for faster translation
df_articles_mob.loc[df_articles_mob['lang']=='hing','Text'] = df_articles_mob[df_articles_mob['lang']=='hing']['Text'].apply(lambda x: x[:3500])

# Hinglish Translation, takes a lot of time due to use of free API
df_articles_mob.loc[df_articles_mob['lang']=='hing','Text'] = utils.translate(df_articles_mob[df_articles_mob['lang']=='hing']['Text'].values, hinglish=True)[0]
#Double translation works better in case of Hinglish sometimes
# Hinglish Translation, takes a lot of time due to use of free API
df_articles_mob.loc[df_articles_mob['lang']=='hing','Text'] = utils.translate(df_articles_mob[df_articles_mob['lang']=='hing']['Text'].values, hinglish=True)[0]

# Hindi translation using Free Google API
df_tweets_mob.loc[df_tweets_mob['lang']=='hi','Text'] = utils.translate(df_tweets_mob[df_tweets_mob['lang']=='hi']['Text'].values)[0]

# Hinglish translation using Free Google API
df_tweets_mob.loc[df_tweets_mob['lang']=='hing','Text'] = utils.translate(df_tweets_mob[df_tweets_mob['lang']=='hing']['Text'].values)[0]
end = time.time()
TRANSLATION_TIME = end-start
print(TRANSLATION_TIME)
# This can be greatly improved by buying a paid API service


290.36538219451904


In [38]:
df_articles_mob.to_pickle('../df_articles_mob_cache.pkl')
df_tweets_mob.to_pickle('../df_tweets_mob_cache.pkl')

In [143]:
df_articles_mob = pd.read_pickle('/content/df_articles_mob_cache.pkl')

In [144]:
df_tweets_mob = pd.read_pickle('/content/df_tweets_mob_cache.pkl')

In [40]:
start = time.time()
headline_generator = headline_generation.headline_gen(device='cuda',path = '/content/AHSG-InterIIT/models/T5-headline.pth')
df_articles_mob['headlines'] = headline_generator.predict(df_articles_mob['Text'].values)
end = time.time()
HEADLINE_GENERATION_TIME = end-start
print(HEADLINE_GENERATION_TIME)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3221: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


60.87309241294861


In [41]:
df_articles_mob[['Text','headlines']]

,Text,headlines
0,A European Central Bank governing council memb...,european central bank governing council warns ...
1,Finance Minister Ms. Nirmala Sitharaman has pr...,budget 2021: 35% increase in capital spending ...
2,"Feb 01, 2021 Shop online for latest, budget-fr...","shop online for latest, budget-friendly smartp..."
3,2nd Watch Taps Global Technology Services Lead...,2nd Watch taps global technology services lead...
4,"French manufacturer, Citroen has finally unvei...",citroen c5 Aircross with 177 horsepower and 40...
...,...,...
66,Motorola released an update to fix one of the ...,home n Motorola Edge+'s smudging problem persists
67,The company has rolled out software update to ...,"if it doesn’t fix the issue, then the phone wi..."
68,Some Xiaomi and Redmi devices are facing a pec...,a fix for airtel users who are stuck in a boot...
69,"Mi, Redmi, and Poco users are facing an issue ...","mi, redmi, and poco users experiencing a bootloop"


In [43]:
df_articles['headline'] = ''
df_articles.loc[df_articles['Mobile_Tech']==1,'headline'] = df_articles_mob['headlines'].values

In [47]:
df_headline_ouput = df_articles[['Text_ID','Mobile_Tech','headline']].copy().reset_index(drop=True)

In [50]:
df_headline_ouput.rename(columns = {'Mobile_Tech':'Mobile_Tech_Flag_Predicted','headline':'Headline_Generated_Eng_Lang'}, inplace = True)

In [53]:
df_headline_ouput.to_csv('../headline-output.csv',index=False)

In [142]:
# PREPROCESS3 for passing data to sentiment pipeline
start = time.time()
article_brand_sent_dicts = []
for row in df_articles_mob[df_articles_mob['num_brands']>0].iterrows():
  text_id = row[1][0]
  text = row[1][1]
  brand_dict = utils.segment_by_rule(text)
  brand_dict['Text_ID'] = text_id
  article_brand_sent_dicts.append(brand_dict)

tweet_brand_sent_dicts = []
for row in df_tweets_mob[df_tweets_mob['num_brands']>0].iterrows():
  text_id = row[1][0]
  text = row[1][1]
  tweet_dict = dict()
  tweet_dict['Text_ID'] = text_id
  tweet_dict['Text'] = text
  tweet_brand_sent_dicts.append(tweet_dict)
end = time.time()
PREPROCESS3_TIME = end - start
print(PREPROCESS3_TIME)

1.1213901042938232


In [184]:
with open('../article_brand_sent_dicts.pkl', 'rb') as f:
  article_brand_sent_dicts = pickle.load(f)
with open('../tweet_brand_sent_dicts.pkl', 'rb') as f:
  tweet_brand_sent_dicts = pickle.load(f)

In [157]:
sentiment = sentiment_inference.SentimentClassifier(bert_path="/content/AHSG-InterIIT/models/mbert-for-sentiment.pth")

In [185]:
start = time.time()
article_sents_out = sentiment.predict(article_brand_sent_dicts,is_tweets=False)
tweets_sents_out = sentiment.predict(tweet_brand_sent_dicts,is_tweets=True)
end = time.time()
SENTIMENT_TIME = end - start
print(SENTIMENT_TIME)

3.633354902267456


In [160]:
 df_articles.loc[df_articles['Text_ID']=='article_4008']

,Text_ID,Text,brands,num_brands,Mobile_Tech
7,article_4008,A European Central Bank governing council memb...,"[google, apple]",2,1


In [165]:
df_tweets['sent_dict'] = [{}]*len(df_tweets)
for d in article_sents_out:
  text_id = d.pop('Text_ID')
  print(d)
  df_articles.loc[df_articles['Text_ID']==text_id]['sent_dict'] = [d]

df_articles['sent_dict'] = [{}]*len(df_articles)
for d in tweets_sents_out:
  text_id = d.pop('Text_ID')
  df_tweets.loc[df_tweets['Text_ID']==text_id,'sent_dict'] = [d]

{'google': 1, 'apple': 1}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


{'lenovo': 2, 'dell': 1, 'acer': 1}
{'nokia': 1, 'realme': 1, 'oppo': 2, 'apple': 1, 'redmi': 1, 'xiaomi': 1, 'samsung': 2, 'vivo': 2}
{'google': 1, 'microsoft': 1}
{'apple': 1}
{'google': 1, 'apple': 1, 'microsoft': 2}
{}
{'moto': 2, 'xiaomi': 2, 'mi': 1}
{'apple': 2}
{'poco': 2, 'redmi': 1, 'realme': 1, 'google': 1}
{'samsung': 2}
{'sony': 1, 'google': 1, 'microsoft': 1}
{'mi': 1}
{'itel': 2}
{'google': 1, 'nvidia': 1}
{'infinix': 1}
{'samsung': 2, 'xiaomi': 2, 'mi': 1}
{'mi': 1}
{'samsung': 1}
{'nokia': 1, 'realme': 2, 'oppo': 1, 'xiaomi': 1, 'samsung': 2, 'mi': 1}
{'apple': 1}
{'poco': 1}
{'lg': 1, 'redmi': 1, 'poco': 1, 'samsung': 1}
{'sony': 2, 'google': 2, 'microsoft': 1, 'nvidia': 1}
{'apple': 1, 'redmi': 1, 'sony': 1, 'poco': 2, 'motorola': 1, 'xiaomi': 1, 'samsung': 2, 'mi': 1}
{'google': 1}
{'poco': 1}
{'xiaomi': 2, 'mi': 1}
{'vivo': 1}
{'infinix': 1}
{'nokia': 1}
{'realme': 1}
{'tecno': 1, 'moto': 1, 'redmi': 1, 'vivo': 1}
{'samsung': 1, 'oneplus': 1}
{'moto': 1, 'oppo': 1,

In [191]:
a = 'Text_ID'
b = 'Mobile_Tech_Flag'
c = 'Brands_Entity_Identified'
d = 'Sentiment_Identified'
df_sentiment_output = []
for row in df_tweets.iterrows():
  text_id = row[1][0]
  mob = row[1][4]
  brandlist = row[1][2]
  sent_dict = row[1][5]
  if len(brandlist)==0:
     df_sentiment_output.append({a:text_id,b:mob,c:'',d:''})
  else:
    for brandname in brandlist:
      brandsent = 'Neutral'
      if brandname in sent_dict.keys():
        if sent_dict[brandname] == 0:
          brandsent = 'Negative'
        elif sent_dict[brandname] == 2:
          brandsent = 'Positive'
      df_sentiment_output.append({a:text_id,b:mob,c:brandname,d:brandsent})

for row in df_articles.iterrows():
  text_id = row[1][0]
  mob = row[1][4]
  brandlist = row[1][2]
  sent_dict = row[1][5]
  if len(brandlist)==0:
    df_sentiment_output.append({a:text_id,b:mob,c:'',d:''})
  else:
    for brandname in brandlist:
      brandsent = 'Neutral'
      if brandname in sent_dict.keys():
        if sent_dict[brandname] == 0:
          brandsent = 'Negative'
        elif sent_dict[brandname] == 2:
          brandsent = 'Positive'
      df_sentiment_output.append({a:text_id,b:mob,c:brandname,d:brandsent})


In [192]:
df_sentiment_output

[{'Brands_Entity_Identified': '',
  'Mobile_Tech_Flag': 0,
  'Sentiment_Identified': '',
  'Text_ID': 'tweet_4001'},
 {'Brands_Entity_Identified': '',
  'Mobile_Tech_Flag': 0,
  'Sentiment_Identified': '',
  'Text_ID': 'tweet_4002'},
 {'Brands_Entity_Identified': '',
  'Mobile_Tech_Flag': 0,
  'Sentiment_Identified': '',
  'Text_ID': 'tweet_4003'},
 {'Brands_Entity_Identified': '',
  'Mobile_Tech_Flag': 0,
  'Sentiment_Identified': '',
  'Text_ID': 'tweet_4004'},
 {'Brands_Entity_Identified': '',
  'Mobile_Tech_Flag': 0,
  'Sentiment_Identified': '',
  'Text_ID': 'tweet_4005'},
 {'Brands_Entity_Identified': '',
  'Mobile_Tech_Flag': 0,
  'Sentiment_Identified': '',
  'Text_ID': 'tweet_4006'},
 {'Brands_Entity_Identified': '',
  'Mobile_Tech_Flag': 0,
  'Sentiment_Identified': '',
  'Text_ID': 'tweet_4007'},
 {'Brands_Entity_Identified': '',
  'Mobile_Tech_Flag': 0,
  'Sentiment_Identified': '',
  'Text_ID': 'tweet_4008'},
 {'Brands_Entity_Identified': '',
  'Mobile_Tech_Flag': 0,
  'Se

In [193]:
df_sentiment_output = pd.DataFrame(df_sentiment_output)

In [194]:
df_sentiment_output

,Text_ID,Mobile_Tech_Flag,Brands_Entity_Identified,Sentiment_Identified
0,tweet_4001,0,,
1,tweet_4002,0,,
2,tweet_4003,0,,
3,tweet_4004,0,,
4,tweet_4005,0,,
...,...,...,...,...
552,article_4224,1,samsung,Neutral
553,article_4224,1,xiaomi,Neutral
554,article_4224,1,oneplus,Neutral
555,article_4224,1,vivo,Neutral


In [197]:
df_sentiment_output.to_csv('/content/sample_data/sentiment-output.csv',index=False)

In [186]:
RUNTIME = PREPROCESS_TIME \
  + BINARY_CLASSIFICATION_TIME \
  + LANGUAGE_DETECTION_TIME \
  + PREPROCESS2_TIME \
  + TRANSLATION_TIME \
  + HEADLINE_GENERATION_TIME \
  + PREPROCESS3_TIME \
  + SENTIMENT_TIME \

In [187]:
print('Total Runtime:',RUNTIME)

Total Runtime: 361.9763469696045
